In [10]:
#%matplotlib widget
import matplotlib.pyplot as plt 

import ipywidgets as widgets
import numpy as np
import os
import pickle
import sys, math
from glob import glob

import colour
from colour_hdri.plotting import plot_tonemapping_operator_image
colour.plotting.colour_style()
colour.utilities.describe_environment();

#import cv2 as cv
import skimage
import imageio

from astropy.io import fits
from colour.models import RGB_COLOURSPACES, RGB_luminance

colorspace = RGB_COLOURSPACES['sRGB']

import astrobf
from astrobf.utils import mask_utils
from astrobf.utils.mask_utils import *

from matplotlib import patches
from matplotlib.collections import PatchCollection

import statmorph
import re


*                                                                             *
*   Interpreter :                                                             *
*       python : 3.8.5 (default, Sep  4 2020, 07:30:14)                       *
*                [GCC 7.3.0]                                                  *
*                                                                             *
*   colour-science.org :                                                      *
*       colour : 0.3.16                                                       *
*       colour-hdri : 0.1.8                                                   *
*                                                                             *
*   Runtime :                                                                 *
*       imageio : 2.9.0                                                       *
*       matplotlib : 3.3.2                                                    *
*       networkx : 2.5                  

In [2]:
dataset = ['EFIFI','Nair'][0]
basedir = ['../../bf_data/EFIGI_catalog/','../../bf_data/Nair_and_Abraham_2010/'][0]
fitsdir = basedir + ['fits_temp_Jan_19/','fits_temp_Dec_28/'][0]

out_dir = basedir+'out1/'

#wdir = '../../OBSdata/efigi-1.6/ima_r/'
fns_g = glob(fitsdir+"*/*g.fits")
fns_r = glob(fitsdir+"*/*r.fits")
fns_i = glob(fitsdir+"*/*i.fits")

fns_g.sort()
fns_r.sort()
fns_i.sort()

eps = 1e-6

In [3]:
def extract_gid(g_path):
    return int(re.split('(\d+)',g_path.split('/')[-2])[1])

gids = [extract_gid(fn) for fn in fns_r]

# 2. Build a mask (GMM)

In [4]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

## Can image sizes vary? probably not. 
-> I don't thnk all of thousands of features are invariant to the image size, even though only empty pixels are cut.

In [5]:
sub_rows = 3
eps = 1e-5

# StatMorph

## Need Image, Mask, Weight, and PSF

In [5]:
import re

fields = [
    'xc_centroid', 'yc_centroid', 
    'ellipticity_centroid', 'elongation_centroid', 'orientation_centroid',
    'xc_asymmetry', 'yc_asymmetry',
    'ellipticity_asymmetry', 'elongation_asymmetry', 'orientation_asymmetry',
    'rpetro_circ', 'rpetro_ellip', 'rhalf_circ', 'rhalf_ellip',
    'r20', 'r80',
    'gini', 'm20', 'gini_m20_bulge', 'gini_m20_merger',
    'sn_per_pixel', 'concentration', 'asymmetry', 'smoothness',
    'sersic_xc', 'sersic_yc', 'sersic_amplitude', 'sersic_rhalf', 'sersic_n', 'sersic_ellip', 'sersic_theta',
    'sky_mean', 'sky_median', 'sky_sigma',
    'flag', 'flag_sersic']

In [ ]:
#ngal = len(fns_r)
#result_arr = np.zeros(ngal, dtype=[('id',int),('ttype',int)]+[(ff,float) for ff in fields])
good_list = glob("/home/hoseung/Work/tonemap/bf_data/EFIGI_catalog/out1/stat_png/good/*.png")
good_gids=[]
for gg in good_list:
    good_gids.append(int(gg.split("/")[-1].split('New')[1].split("_")[0]))                    

good_gids.sort()

print(good_gids)

In [101]:
morphs=[]
for i, fn in enumerate(fns_r):
    if i < 2500:
        continue
    if i % 500 == 499:
        pickle.dump(morphs, open(out_dir+f"Morphs/final_morphs{i}.pickle", "wb"),
                   protocol=pickle.HIGHEST_PROTOCOL)
        morphs=[]
        print(i)

    img_name = fn.split("/")[-2]
    gid = int(re.split('(\d+)',img_name)[1])
    if gid not in good_gids:
        continue
    if dataset=="Nair": img_name = img_name.split('.')[0]
    hdulist = fits.open(fn)
    # Ensure pixel values are positive
    img = hdulist[0].data
    img -= (img.min() - eps) 
    
    mask = pickle.load(open(out_dir+f"/masks/{img_name}_mask.pickle", 'rb'))
    
    weight = fits.open(fn.replace(".fits", ".weight.fits"))[0].data
    subtracted = img.copy()
    subtracted[~mask] = 0

    morph = statmorph.source_morphology(
        subtracted, mask, weightmap=weight)[0]#, gain=4000)[0] ##
    hdulist.close()
    morph._gid = gid
    morphs.append(morph)
pickle.dump(morphs, open(out_dir+f"Morphs/final_morphs{i}.pickle", "wb"),
            protocol=pickle.HIGHEST_PROTOCOL)

/home/hoseung/anaconda3/envs/tm38/lib/python3.8/site-packages/astropy/modeling/functional_models.py:2450: RuntimeWarning: overflow encountered in power
  return amplitude * np.exp(-bn * (z ** (1 / n) - 1))


2999


3499


3999


In [6]:
all_morphs = glob(out_dir+"Morphs/final_morphs*.pickle")
all_morphs.sort()

In [7]:
from statmorph.utils import image_diagnostics
#import make_figure

In [8]:
all_morphs

['../../bf_data/EFIGI_catalog/out1/Morphs/final_morphs1499.pickle',
 '../../bf_data/EFIGI_catalog/out1/Morphs/final_morphs1999.pickle',
 '../../bf_data/EFIGI_catalog/out1/Morphs/final_morphs2499.pickle',
 '../../bf_data/EFIGI_catalog/out1/Morphs/final_morphs2999.pickle',
 '../../bf_data/EFIGI_catalog/out1/Morphs/final_morphs3499.pickle',
 '../../bf_data/EFIGI_catalog/out1/Morphs/final_morphs3999.pickle',
 '../../bf_data/EFIGI_catalog/out1/Morphs/final_morphs4457.pickle',
 '../../bf_data/EFIGI_catalog/out1/Morphs/final_morphs499.pickle',
 '../../bf_data/EFIGI_catalog/out1/Morphs/final_morphs999.pickle']

In [11]:
gids = np.array(gids)
#result_arr = np.zeros(ngal, dtype=[('id',int),('ttype',int)]+[(ff,float) for ff in fields])
for alm in all_morphs[3:7]:
    print(alm)
    mps = pickle.load(open(alm,'rb'))
    for morph in mps:
        fn = fns_r[np.where(gids==morph._gid)[0].squeeze()]
        # Double Check
        img_name = fn.split("/")[-2]
        gid = int(re.split('(\d+)',img_name)[1])
        assert gid == morph._gid
            
        hdulist = fits.open(fn)
        img = hdulist[0].data
        img -= (img.min() - eps)
        
        image_diagnostics.make_figure(morph, nrows=3,
                                      savefig=out_dir+f'stat_png/final_{morph._gid}_summary.png',
                                      img_org=None)

../../bf_data/EFIGI_catalog/out1/Morphs/final_morphs2999.pickle
../../bf_data/EFIGI_catalog/out1/Morphs/final_morphs3499.pickle
../../bf_data/EFIGI_catalog/out1/Morphs/final_morphs3999.pickle
../../bf_data/EFIGI_catalog/out1/Morphs/final_morphs4457.pickle


종종 fit 실패함. 이상한 object가 많으니 충분히 그럴 수 있긴 함. 눈으로 확인.

'모범적인' Sa, Sb, Sc, Sd가 잘 보이도록만 만들면 됨. fore/backgroud object 때문에 fit이 안 되는 경우는 그냥 무시. 얼마나 자주 그러는지만 체크. 


